In [34]:
import numpy as np
import random
from classes import *
from selection_scheme import *
import matplotlib.pyplot as plt
from itertools import permutations
from tqdm import tqdm

In [35]:
class TSP():
    def __init__(self, filename):
        self.population = []
        self.best_so_far = []
        self.bestpath = node([-2,-2,-2])
        self.avg_so_far = []
        self.children = []
        self.node_lst = []
        self.filename = filename

        self.mutation_rate = 0.7
        self.no_generations = 1000
        self.no_iteration = 1
        self.population_size = 100
        self.off_size = 75

    def generate_population(self):
        for i in range(self.no_generations):
            random.shuffle(self.node_lst)
            temp = copy.deepcopy(self.node_lst)
            temp = chromosome(temp)
            self.population.append(temp)

    def make_children(self):
        for x in range(self.off_size):
            selected_chromosomes = binary_tournament_selection(self.population, 2, True, False)
            temp = selected_chromosomes[0].crossover(selected_chromosomes[1])
            self.children.append(temp)

    def mutate(self):
        for x in range(len(self.children)):
            if random.random() < self.mutation_rate:
                self.children[x].mutate()

    def survival_of_the_fitest(self):
        self.population = self.population + self.children
        self.children.clear()
        self.population = truncation_selection(self.population, self.population_size, False, False)
        
        # sort population by fitness
        self.population.sort(key=lambda x: x.fitness)

        self.best_so_far.append(self.population[0].fitness)
        self.bestpath = self.population[0]
        self.avg_so_far.append(sum([self.population[x].fitness for x in range(len(self.population))])/len(self.population))


In [36]:
class Kmean():
    def __init__(self, filename, k=3, n=10) -> None:
        self.K = k
        self.N = n
        self.filename = filename
        self.dataset = self.initializePoints()
        self.cluster_lst = self.keepClustering(self.dataset, self.K, self.N, False)
        self.dict = self.makeDic()
        self.region = self.cord_to_region()
        

    def initializePoints(self, ):
        points = []
        with open(self.filename, "r") as f:
            for i in range(7):
                f.readline()
            for line in f:
                data = line.split()
                data = [float(x) for x in data[1:]]
                points.append(data)
        return points[:-1]

    def point_clustering(self, points, centroids):
        for point in points:
            nearest_dist_from_ref = 999**999
            centro_point = None
            for i in centroids.keys():
                euclidean_dist = np.linalg.norm(np.subtract(list(i),list(point)))
                if euclidean_dist < nearest_dist_from_ref:
                    nearest_dist_from_ref = euclidean_dist
                    centro_point = i
            centroids[centro_point].append(point)
        return centroids

    def visualize(self, clustered_data, Iteration):
        plotx = []
        ploty = []
        clr = ["cyan","magenta","green", "red", "yellow", "blue"]

        for x in clustered_data.keys():
            try:
                array = np.array(clustered_data[x])
                plotx,ploty = array[:,0], array[:,1]
                plt.plot(plotx,ploty, '+', color=clr.pop())
            except:
                pass

        for x in clustered_data.keys():
            plt.plot(x[0], x[1], 'o' ,color='black')

        plt.title("Iteration: " + str(Iteration))
        plt.show()

    def mean_center(self, centroids):
        new_centers = {}
        for x in centroids.keys():
            array = np.array(centroids[x])
            np.shape(array)
            try:
                xcord,ycord = np.mean(array[:,0]), np.mean(array[:,1])
                gravity_center = (xcord,ycord)
                new_centers[gravity_center] = []
            except:
                gravity_center = x
                new_centers[gravity_center] = []
                self.visualize(centroids, 404)
        return new_centers

    def cluster(self, points,K,visuals = True):
        clusters=[]
        data_cluster = self.point_clustering(points, self.make_random_center(points, K))
        prev_cntr = data_cluster
        iteration = 0
        while(True):
            new_cntr = self.mean_center(data_cluster)
            data_cluster =  self.point_clustering(points, new_cntr)
            if visuals: 
                self.visualize(data_cluster, iteration)

            if prev_cntr.keys() == new_cntr.keys():
                break
            else:
                prev_cntr = data_cluster
            iteration += 1
        
        return data_cluster

    def make_random_center(self, points, k):
        centroids = {}
        lst = random.sample(points, k)
        for x in lst:
            centroids[(x[0],x[1])] = []
        return centroids

    def clusterQuality(self, clusters):
        score = -1
        for x in clusters.keys():
            for y in clusters[x]:
                score += (x[0] - y[0])**2 + (x[1] - y[1])**2

        return score

    def keepClustering(self, points,K,N,visuals):
        clusters = []
        lst = []
        min = 999**999
        for x in range(N):
            print("Iteration No.: ", x+1, "/", N)
            data_cluster = self.cluster(points,K,visuals)
            score = self.clusterQuality(data_cluster)
            print("Score: ", score,"\n")
            if score < min:
                min = score
                clusters = ([data_cluster, min, x+1])

        for x in clusters[0].keys():
            lst.append(clusters[0][x])
        return lst

    def makeDic(self):
        dic = {}
        with open(self.filename, "r") as f:
            for i in range(7):
                f.readline()
            for line in f:
                data = line.split()
                if len(data) > 1:
                    dic[float(data[1]),float(data[2])] = int(data[0])
        return dic
    
    def cord_to_region(self):
        region = []
        lst = []
        for i in self.cluster_lst:
            for j in i:
                lst.append(self.dict[j[0],j[1]])
            region.append(lst)
            lst = []
        return region


In [37]:
def readfile(filename):
    node_lst = []
    with open(filename, "r") as f:
        for i in range(7):
            f.readline()
        for line in f:
            data = line.split()
            if len(data) > 1:
                temp = node(data)
                node_lst.append(temp)
    return node_lst


In [38]:
def filter_nodes(node_lst, region):
    lst = []
    for y in region:
        mlst = []
        for x in node_lst:
            if x.id in y:
                mlst.append(x)
        lst.append(mlst)
        mlst = []
    return lst


In [39]:
cluster = Kmean("qa194.tsp")

filter_lst = []
node_lst = readfile("qa194.tsp")    
filter_lst = filter_nodes(node_lst, cluster.region)

tsp1 = TSP("qa194.tsp")
tsp2 = TSP("qa194.tsp")
tsp3 = TSP("qa194.tsp")

tsp1.node_lst = filter_lst[0]
tsp2.node_lst = filter_lst[1]
tsp3.node_lst = filter_lst[2]

tsp1.generate_population()
tsp2.generate_population()
tsp3.generate_population()

Iteration No.:  1 / 10
Score:  7905971.23055232 

Iteration No.:  2 / 10
Score:  7905971.230552321 

Iteration No.:  3 / 10
Score:  7903655.814527427 

Iteration No.:  4 / 10
Score:  7905971.230552324 

Iteration No.:  5 / 10
Score:  7905971.230552323 

Iteration No.:  6 / 10
Score:  7905971.230552321 

Iteration No.:  7 / 10
Score:  7905971.23055232 

Iteration No.:  8 / 10
Score:  7903655.814527425 

Iteration No.:  9 / 10
Score:  7903655.81452743 

Iteration No.:  10 / 10
Score:  7903655.814527424 



In [40]:
for i in tqdm(range(tsp1.no_generations)):
    tsp1.make_children()
    tsp1.mutate()
    tsp1.survival_of_the_fitest()
    if i % 100 == 0:
        print("TSP1: Generation No.: ", i+1)
        print("Best Fitness Soo Far: ", tsp1.best_so_far[-1])
    
print("TSP1: Best Fitness Soo Far: ", tsp1.bestpath.fitness)
print("TSP1: Best Path: ", tsp1.bestpath.get_sequence())

  0%|          | 2/1000 [00:00<01:49,  9.12it/s]

TSP1: Generation No.:  1
Best Fitness Soo Far:  9415.86099689006


 10%|█         | 102/1000 [00:11<01:34,  9.50it/s]

TSP1: Generation No.:  101
Best Fitness Soo Far:  5448.181360357607


 20%|██        | 201/1000 [00:21<01:44,  7.62it/s]

...

TSP1: Generation No.:  801
Best Fitness Soo Far:  5005.078760186081


 90%|█████████ | 902/1000 [01:35<00:10,  9.64it/s]

TSP1: Generation No.:  901
Best Fitness Soo Far:  5005.078760186081


100%|██████████| 1000/1000 [01:45<00:00,  9.48it/s]

TSP1: Best Fitness Soo Far:  5005.078760186081
TSP1: Best Path:  [1, 6, 13, 23, 25, 76, 71, 59, 36, 94, 90, 89, 98, 85, 86, 99, 111, 104, 114, 113, 109, 80, 7, 2, 4, 8, 16, 62, 82, 87, 101, 63, 65, 20, 11, 17, 14]


In [41]:
for i in tqdm(range(tsp2.no_generations)):
    tsp2.make_children()
    tsp2.mutate()
    tsp2.survival_of_the_fitest()
    if i % 100 == 0:
        print("TSP2: Generation No.: ", i+1)
        print("Best Fitness Soo Far: ", tsp2.best_so_far[-1])

print("TSP2: Best Fitness Soo Far: ", tsp2.bestpath.fitness)
print("TSP2: Best Path: ", tsp2.bestpath.get_sequence())

  0%|          | 2/1000 [00:00<03:37,  4.59it/s]

TSP2: Generation No.:  1
Best Fitness Soo Far:  13754.038348052622


 10%|█         | 101/1000 [00:21<03:21,  4.45it/s]

TSP2: Generation No.:  101
Best Fitness Soo Far:  8502.970383935615


 20%|██        | 202/1000 [00:43<02:42,  4.92it/s]

...

TSP2: Generation No.:  801
Best Fitness Soo Far:  6407.177042109842


 90%|█████████ | 902/1000 [03:10<00:21,  4.65it/s]

TSP2: Generation No.:  901
Best Fitness Soo Far:  6407.177042109842


100%|██████████| 1000/1000 [03:30<00:00,  4.74it/s]

TSP2: Best Fitness Soo Far:  6407.177042109842
TSP2: Best Path:  [156, 184, 175, 173, 174, 187, 183, 179, 192, 145, 130, 127, 132, 134, 119, 139, 138, 122, 128, 133, 151, 158, 180, 181, 178, 170, 168, 167, 162, 131, 136, 141, 144, 149, 142, 137, 126, 125, 157, 186, 190, 194, 182, 176, 189, 191, 188, 165, 159, 129, 135, 147, 152, 161, 169, 163, 164, 150, 153, 154, 146, 140, 172, 177, 193, 171, 155, 148, 143, 160, 166, 185]


In [42]:
for i in tqdm(range(tsp3.no_generations)):
    tsp3.make_children()
    tsp3.mutate()
    tsp3.survival_of_the_fitest()
    if i % 100 == 0:
        print("TSP3: Generation: ", i+1)
        print("Best Fitness Soo Far: ", tsp3.best_so_far[-1])

print("TSP3: Best Fitness Soo Far: ", tsp3.bestpath.fitness)
print("TSP3: Best Path: ", tsp3.bestpath.get_sequence())

  0%|          | 1/1000 [00:00<04:37,  3.60it/s]

TSP3: Generation:  1
Best Fitness Soo Far:  15427.253970860595


 10%|█         | 101/1000 [00:25<03:35,  4.16it/s]

TSP3: Generation:  101
Best Fitness Soo Far:  10051.898102444322


 20%|██        | 201/1000 [00:50<03:12,  4.15it/s]

...

TSP3: Generation:  801
Best Fitness Soo Far:  10051.898102444322


 90%|█████████ | 901/1000 [03:43<00:25,  3.93it/s]

TSP3: Generation:  901
Best Fitness Soo Far:  10051.898102444322


100%|██████████| 1000/1000 [04:07<00:00,  4.04it/s]

TSP3: Best Fitness Soo Far:  10051.898102444322
TSP3: Best Path:  [42, 54, 68, 45, 43, 34, 21, 18, 37, 47, 51, 58, 22, 19, 110, 123, 118, 106, 112, 96, 75, 74, 79, 57, 73, 41, 61, 30, 31, 60, 93, 95, 88, 120, 117, 77, 67, 12, 46, 32, 9, 5, 29, 48, 49, 35, 53, 105, 107, 66, 50, 56, 15, 78, 81, 91, 102, 69, 70, 44, 84, 64, 33, 3, 10, 24, 26, 72, 92, 97, 83, 52, 55, 38, 39, 28, 27, 40, 116, 100, 108, 103, 124, 121, 115]


In [43]:
final = TSP("qa194.tsp")
lst = []
lst.append(tsp1.bestpath.sequence)
lst.append(tsp2.bestpath.sequence)
lst.append(tsp3.bestpath.sequence)

final.node_lst = tsp1.bestpath.sequence +tsp2.bestpath.sequence + tsp3.bestpath.sequence
perm = list(permutations(lst))
for x in perm:
    temp = []
    for y in x:
        temp += y
    final.population.append(chromosome(temp))
    temp = []


In [44]:
for i in tqdm(range(final.no_generations)):
    final.make_children()
    final.mutate()
    final.survival_of_the_fitest()
    if i % 100 == 0:
        print("Final: Generation: ", i+1)
        print("Best Fitness Soo Far: ", final.bestpath.fitness)

print("Final: Best Fitness Soo Far: ", final.bestpath.fitness)
print("Final: Best Path: ", final.bestpath.get_sequence())

  0%|          | 1/1000 [00:00<11:29,  1.45it/s]

Final: Generation:  1
Best Fitness Soo Far:  22785.5124512993


 10%|█         | 101/1000 [00:57<08:31,  1.76it/s]

Final: Generation:  101
Best Fitness Soo Far:  22097.979490790694


 20%|██        | 201/1000 [01:55<07:34,  1.76it/s]

...

Final: Generation:  801
Best Fitness Soo Far:  21251.912373465682


 90%|█████████ | 901/1000 [08:35<00:56,  1.76it/s]

Final: Generation:  901
Best Fitness Soo Far:  21251.912373465682


100%|██████████| 1000/1000 [09:31<00:00,  1.75it/s]

Final: Best Fitness Soo Far:  21251.912373465682
Final: Best Path:  [1, 6, 13, 23, 25, 76, 71, 59, 36, 94, 90, 89, 98, 85, 86, 99, 111, 104, 114, 113, 109, 80, 7, 2, 4, 8, 16, 62, 82, 87, 101, 63, 65, 20, 11, 17, 14, 42, 54, 68, 45, 34, 21, 18, 37, 47, 51, 58, 19, 110, 123, 124, 106, 112, 96, 75, 74, 79, 57, 73, 41, 61, 30, 27, 29, 22, 60, 93, 95, 88, 115, 117, 77, 67, 46, 32, 12, 9, 5, 31, 48, 49, 35, 53, 105, 100, 66, 50, 56, 15, 70, 81, 91, 102, 69, 78, 44, 38, 64, 33, 3, 10, 24, 26, 72, 92, 97, 83, 52, 55, 40, 39, 28, 43, 84, 116, 107, 108, 103, 118, 121, 120, 157, 184, 175, 173, 174, 187, 183, 179, 192, 145, 130, 127, 132, 134, 119, 139, 138, 122, 128, 133, 151, 162, 180, 181, 178, 170, 168, 167, 158, 131, 136, 141, 144, 149, 142, 137, 126, 125, 172, 186, 190, 194, 182, 176, 189, 191, 188, 165, 159, 129, 135, 147, 152, 169, 161, 163, 164, 150, 153, 154, 146, 140, 156, 177, 193, 171, 155, 148, 143, 160, 166, 185]
